# Main

In [13]:
%pwd

'/teamspace/studios/this_studio/endoscopic'

In [14]:
root_dir = '/teamspace/studios/this_studio/dataset/labeled-images'
ckpt_path = './checkpoints'
csv_result_path = f'{ckpt_path}/result.csv'

os.makedirs(ckpt_path, exist_ok=True)
os.makedirs(f'{ckpt_path}/descriptors/', exist_ok=True)
os.makedirs(f'{ckpt_path}/codebook/', exist_ok=True)
os.makedirs(f'{ckpt_path}/embedding_df/', exist_ok=True)
os.makedirs(f'{ckpt_path}/best_model/', exist_ok=True)
# os.makedirs(f'{ckpt_path}/{csv_result_path}', exist_ok=True)

from itertools import product

#  Define search space
hyperparam_grid = {
    "feature_extractor": ["SIFT", "SURF"],
    "strongest_percent": [1, 0.7, 0.8],
    "clustering_algorithm": ["KMeans", "BIRCH"],
    "vector_size": [128, 64],
}
# Generate all combinations of hyperparameters
hyperparam_combinations = list(product(*hyperparam_grid.values()))
hyperparam_keys = list(hyperparam_grid.keys())


# Iterate over all hyperparameter combinations
for combination in hyperparam_combinations:
    # Create a dictionary of current hyperparameter combination
    hyperparams = dict(zip(hyperparam_keys, combination))
    break

hyperparams

{'feature_extractor': 'SIFT',
 'strongest_percent': 1,
 'clustering_algorithm': 'KMeans',
 'vector_size': 128}

In [3]:
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
import joblib

from BoVW import BagOfVisualWords
# ===========================================================
# Initialization
# ===========================================================
np.random.seed(0)  # reproducibility
root_dir = '/teamspace/studios/this_studio/dataset/labeled-images'
ckpt_path = './checkpoints'
csv_result_path = f'{ckpt_path}/result.csv'

# os.makedirs(ckpt_path, exist_ok=True)
# os.makedirs(f'{ckpt_path}/descriptors/', exist_ok=True)
# os.makedirs(f'{ckpt_path}/codebook/', exist_ok=True)
# os.makedirs(f'{ckpt_path}/embedding_df/', exist_ok=True)
# os.makedirs(f'{ckpt_path}/best_model/', exist_ok=True)

s_id = np.random.randint(0, 100)

# ===========================================================
# HYPERPARAMs SWEEPING
# ===========================================================
'''hyperparameters'''
# descriptors extracting
method = hyperparams['feature_extractor']
strongest_percent = hyperparams['strongest_percent']
# quantization
cluster_algorithm = hyperparams['clustering_algorithm']
k = hyperparams['vector_size'] # #cluster = vector size
'''hyperparameters'''

# --------------------------------------------------------------
# ----- 1. extracting descriptors -------------
# --------------------------------------------------------------
model = BagOfVisualWords(
        root_dir=root_dir,
        method=method
    )
# sample_size = len(model.df)
sample_size = 20
descriptors_lake = model.extract_descriptors(sample_size=sample_size,
                                            strongest_percent=strongest_percent)

# saving 
descriptors_lake_path = joblib.dump(descriptors_lake,
                                    f'{ckpt_path}/descriptors/id{s_id}-{sample_size}_img-{model.method}_extractor-{strongest_percent*100}%_strongest.pkl',
                                    compress=3)
del descriptors_lake # free memory
print('='*20, 'Completely extracting descriptors', '='*20)

# =================== load ================================
descriptors_lake = joblib.load(*descriptors_lake_path) # unpack list


# --------------------------------------------------------------
#  2. building codebook (visual vocabulary) 
# --------------------------------------------------------------
codebook = model.build_codebook(descriptors_lake=descriptors_lake,
                                    cluster_algorithm='kmean',
                                    k=200
                                )
# saving 
codebook_path = joblib.dump(codebook,
                            f'{ckpt_path}/codebook/id{s_id}-{cluster_algorithm}_cluster_algorithm-k={k}.pkl',
                            compress=3)
del descriptors_lake # free memory
del codebook # free memory
print('='*20, 'Completely building codebook', '='*20)

# --------------------------------------------------------------
# 3. Embedding representation 
# --------------------------------------------------------------

codebook = joblib.load(*codebook_path) # unpack list
# n_imgs = len(model.df)
n_imgs = 50 # for test

headers = [f'feature{i}' for i in range(codebook.shape[0])]
embedding_df = pd.DataFrame(columns=headers)
labels = []

# Embedding entire dataset
normalized = False
try:
    for idx in tqdm(range(n_imgs)):
        img, label = model._get_item(idx)
        embedding = model.get_embedding(idx, codebook, normalized=normalized)
        # Add a row to the DataFrame
        embedding_df.loc[len(embedding_df)] = embedding
        labels.append(label)
        # break
except Exception as e:
    print(e)
    pass

embedding_df['label'] = pd.Series(labels, dtype='int')

# saving
embedding_df_path = f'{ckpt_path}/embedding_df/id{s_id}-{n_imgs}_img-normalized={normalized}.csv'
embedding_df.to_csv(embedding_df_path, index=False)

del embedding_df # free memory
print('='*20, 'Completely building embedding representation', '='*20)


# --------------------------------------------------------------
# 4. Classification 
# --------------------------------------------------------------
df = pd.read_csv(embedding_df_path)

X = df.drop('label', axis=1)
y = df['label']

from utils import TPOT_autoML, evaluate_model

metadata = os.path.basename(embedding_df_path)
best_model = TPOT_autoML(X, y, ckpt_path, metadata)
# Evaluate the best model using custom metrics
metrics = evaluate_model(X, y, best_model)

## ==================== Accumulating saving final result ======================
results = []
results.append({**hyperparams, **metrics})
# Convert results to a DataFrame and save to CSV
results_df = pd.DataFrame(results)

# If file exists, append; otherwise, create with headers
if csv_result_path:
    try:
        existing_df = pd.read_csv(csv_result_path)
        results_df = pd.concat([existing_df, results_df], ignore_index=True)
    except FileNotFoundError:
        pass

results_df.to_csv(csv_result_path, index=False)

 15%|█▌        | 3/20 [00:00<00:01, 12.78it/s]

100%|██████████| 20/20 [00:02<00:00,  9.01it/s]


==================== Completely extracting descriptors ====================


Initializing: 100%|██████████| 10/10 [00:00<00:00, 11.72it/s]


==================== Completely building codebook ====================


...

Optimization Progress:   0%|          | 0/120 [00:00<?, ?pipeline/s]


Generation 1 - Current best internal CV score: 1.0

Generation 2 - Current best internal CV score: 1.0

Generation 3 - Current best internal CV score: 1.0

Generation 4 - Current best internal CV score: 1.0

Generation 5 - Current best internal CV score: 1.0

Best pipeline: ExtraTreesClassifier(input_matrix, bootstrap=True, criterion=entropy, max_features=0.1, min_samples_leaf=15, min_samples_split=16, n_estimators=100)
Exporting the best model...
Best model saved as 'best_model.pkl'


0it [00:00, ?it/s]/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/sklearn/metrics/_classification.py:386: UserWarning: A single label was found in 'y_true' and 'y_pred'. For the confusion matrix to have the correct shape, use the 'labels' parameter to pass all known labels.
  warnings.warn(
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/sklearn/metrics/_classification.py:386: UserWarning: A single label was found in 'y_true' and 'y_pred'. For the confusion matrix to have the correct shape, use the 'labels' parameter to pass all known labels.
  warnings.warn(
2it [00:00,  9.77it/s]/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/sklearn/metrics/_classification.py:386: UserWarning: A single label was found in 'y_true' and 'y_pred'. For the confusion matrix to have the correct shape, use the 'labels' parameter to pass all known labels.
  warnings.warn(
3it [00:00,  9.45it/s]/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-p

Average Metrics Across All Folds:


Precision (Macro): 1.0
Recall (Macro): 1.0
F1-Score (Macro): 1.0


Precision (Micro): 1.0
Recall (Micro): 1.0
F1-Score (Micro): 1.0


Balanced Accuracy: 1.0


NameError: name 'metrics' is not defined

# Draft

In [5]:
metrics = evaluate_model(X, y, b_model)

## ==================== Accumulating saving final result ======================
results = []
results.append({**hyperparams, **metrics})
# Convert results to a DataFrame and save to CSV
results_df = pd.DataFrame(results)

# If file exists, append; otherwise, create with headers
if csv_result_path:
    try:
        existing_df = pd.read_csv(csv_result_path)
        results_df = pd.concat([existing_df, results_df], ignore_index=True)
    except FileNotFoundError:
        pass

results_df.to_csv(csv_result_path, index=False)

0it [00:00, ?it/s]/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/sklearn/metrics/_classification.py:386: UserWarning: A single label was found in 'y_true' and 'y_pred'. For the confusion matrix to have the correct shape, use the 'labels' parameter to pass all known labels.
  warnings.warn(
1it [00:00,  7.91it/s]/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/sklearn/metrics/_classification.py:386: UserWarning: A single label was found in 'y_true' and 'y_pred'. For the confusion matrix to have the correct shape, use the 'labels' parameter to pass all known labels.
  warnings.warn(
2it [00:00,  8.85it/s]/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/sklearn/metrics/_classification.py:386: UserWarning: A single label was found in 'y_true' and 'y_pred'. For the confusion matrix to have the correct shape, use the 'labels' parameter to pass all known labels.
  warnings.warn(
3it [00:00,  9.11it/s]/home/zeus/miniconda3/envs/cloudspace

Average Metrics Across All Folds:


Precision (Macro): 1.0
Recall (Macro): 1.0
F1-Score (Macro): 1.0


Precision (Micro): 1.0
Recall (Micro): 1.0
F1-Score (Micro): 1.0


Balanced Accuracy: 1.0


In [ ]:
from utils import TPOT_autoML

metadata = os.path.basename(embedding_df_path)
b_model = TPOT_autoML(X, y, ckpt_path, metadata)
# Evaluate the best model using custom metrics
evaluate_model(X, y, b_model)